In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import glob
import re
import os

In [ ]:
df = pd.read_csv("/Users/jeongjuhyeon/Desktop/Python/CS230/data/5_pap_NOhaptics_10.csv")

In [35]:
data_path_lst = pd.read_csv("/Users/jeongjuhyeon/Desktop/Python/CS230/data/Participant_6_results.csv")
data_path_lst

,Condition,Subcondition,Trial,Sensor1 Mean,Sensor2 Mean
0,PAP,HAPTICS,1,1132.18,1133.91
1,PAP,HAPTICS,2,1148.16,1148.12
2,PAP,HAPTICS,3,1125.18,1119.08
3,PAP,HAPTICS,4,1106.45,1106.37
4,PAP,HAPTICS,5,1161.47,1157.92
5,PAP,HAPTICS,6,1130.36,1129.46
6,PAP,HAPTICS,7,1197.76,1192.59
7,PAP,HAPTICS,8,1143.66,1187.48
8,PAP,HAPTICS,9,1149.43,1142.73
9,PAP,HAPTICS,10,1119.73,1121.87


In [54]:

all_files = glob.glob("data/*.csv")
datasets = {f: pd.read_csv(f) for f in all_files}


labels = []
examples = []
for fname, df in datasets.items():
    # .csv 확장자 제거
    base = os.path.basename(fname).replace(".csv", "")
    # print(f"file name: {base}")
    if len(base.split("_")) ==4:
        _, Condition, Subcondition,Trial  = base.split("_")
        if "pap" in Condition:
            Condition = Condition.replace("pap", "PAP")
        if "camera" in Condition:
            Condition = Condition.replace('camera', 'Camera')
        # print(Condition,Subcondition, Trial)
        row = data_path_lst[
        (data_path_lst["Condition"] == Condition)
        & (data_path_lst["Subcondition"] == Subcondition)
        & (data_path_lst["Trial"] == Trial)]
        
        if row.empty:
            print(f'no data {base}')
        


        if "Device 0 - Follower 0" in df.columns and "Device 0 - Follower 1" in df.columns:
            example = df[["Device 0 - Follower 0", "Device 0 - Follower 1"]].copy()
            label = row[["Sensor1 Mean", "Sensor2 Mean"]].values.flatten().tolist()
            examples.append(example)
            labels.append(label)



print(f"총 {len(examples)}개의 example이 생성되었습니다.")
print(examples[0])  



no data 3_training_NOhaptics_2
총 41개의 example이 생성되었습니다.
     Device 0 - Follower 0  Device 0 - Follower 1
0                     1124                   1119
1                     1124                   1118
2                     1124                   1118
3                     1123                   1118
4                     1124                   1118
..                     ...                    ...
823                   1017                    973
824                   1027                    977
825                   1015                    979
826                   1018                    972
827                    997                    970

[828 rows x 2 columns]


In [3]:
class PressureDataset(Dataset):
    def __init__(self, input, output, seq_len=None):
        """
        Args:
            csv_path (str): CSV file path
            seq_len (int, optional): the lenth of sequence 
        """
        input_data = input
        output_data = output
        
        # pressure = df['pressure'].values
        self.x_data = torch.tensor(input_data, dtype=torch.float32).unsqueeze(-1)  # (T, 1)

        if seq_len is not None:
            self.x_data = self.x_data[:seq_len]

        
        self.y_data = torch.tensor([output_data], dtype=torch.float32)

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return self.x_data, self.y_data

class PressureRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, (h_n, c_n) = self.rnn(x)
        #  hidden state h_n  (num_layers, batch, hidden_size)
        h_last = h_n[-1]  # (batch, hidden_size)
        y_pred = self.fc(h_last)
        return y_pred



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_data = df["Device 0 - Follower 2"].fillna(method="ffill")
output_data = np.array([1130.31])

dataset = PressureDataset(input_data,output_data)
loader = DataLoader(dataset, batch_size=1, shuffle=False)

model = PressureRNN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(200):
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)  
        loss = criterion(pred, y.squeeze(-1))
        loss.backward()
        optimizer.step()
    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/200], Loss: {loss.item():.6f}")